In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import os

In [ ]:
import sqlite3
import pandas as pd

# Connecter à la base de données SQLite
conn = sqlite3.connect('basketball.sqlite')

# Obtenir la liste des tables dans la base de données
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Parcourir chaque table et l'exporter en CSV
for table_name in tables['name']:
    # Lire la table dans un DataFrame pandas
    df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
    
    # Exporter la table en fichier CSV
    df.to_csv(f'{table_name}.csv', index=False)
    print(f'La table {table_name} a été exportée en {table_name}.csv')

# Fermer la connexion à la base de données
conn.close()


## Présentation générale des équipes et du nombres de games

In [51]:
# Charger le fichier CSV avec les bonnes options
df = pd.read_csv('csv\Game.csv', dtype=str, low_memory=False)

# Convertir les colonnes numériques appropriées
df['GAME_ID'] = df['GAME_ID'].astype(int)
df['TEAM_ID_HOME'] = df['TEAM_ID_HOME'].astype(int)
df['TEAM_ID_AWAY'] = df['TEAM_ID_AWAY'].astype(int)
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Ajouter une colonne pour la saison (extraction de l'année)
df['season'] = df['GAME_DATE'].dt.year

# Exclure la saison 2021
df = df[df['season'] != 2021]

# Compter le nombre d'équipes par saison
team_count = df.groupby('season')['TEAM_ID_HOME'].nunique().reset_index(name='total_team_count')

# Compter le nombre de jeux par saison
game_count = df.groupby('season')['GAME_ID'].nunique().reset_index(name='total_game_count')

# Fusionner les deux décomptes
team_and_game_count = pd.merge(team_count, game_count, on='season')

# Créer une visualisation avec deux axes y
two_y_axis_plot = make_subplots(specs=[[{"secondary_y": True}]])
team_count_trace = px.bar(team_and_game_count, x="season", y="total_team_count")
game_count_trace = px.line(team_and_game_count, x="season", y="total_game_count")

team_count_trace.update_traces(name="Total des équipes", showlegend=True, opacity=0.6)
game_count_trace.update_traces(name="Total des games", showlegend=True, line_color="red")

two_y_axis_plot.add_trace(team_count_trace.data[0], secondary_y=False)
two_y_axis_plot.add_trace(game_count_trace.data[0], secondary_y=True)

two_y_axis_plot.update_yaxes(title_text="Total des équipes", secondary_y=False)
two_y_axis_plot.update_yaxes(title_text="Total des games", secondary_y=True)
two_y_axis_plot.update_layout(title_text="1946 à 2020 : Tendance du nombre total d'équipes et de games NBA")

two_y_axis_plot.show()


<>:2: SyntaxWarning:

invalid escape sequence '\G'

<>:2: SyntaxWarning:

invalid escape sequence '\G'

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_18068\1217085995.py:2: SyntaxWarning:

invalid escape sequence '\G'



## L'évolution de la draft depuis 1946

In [36]:
# Charger le fichier CSV
df_draft = pd.read_csv('./csv/Draft.csv')

# Compter le nombre total de joueurs draftés par année
year_draft_total = df_draft.groupby('yearDraft').size().reset_index(name='total_drafted')

# Créer le graphique
fig = px.line(year_draft_total, 
              x="yearDraft", y="total_drafted", 
              title='Tendance des drafts NBA de 1949 à 2020')

# Mettre à jour les légendes en français
fig.update_layout(
    xaxis_title='Année de draft',
    yaxis_title='Total des joueurs draftés',
    title='Tendance des drafts NBA de 1949 à 2020'
)

fig.show()


### Origine des joueurs professionnels

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Charger le fichier CSV
df_draft = pd.read_csv('./csv/Draft.csv')

# Agréger les données nécessaires pour le graphique en secteurs
draft_player_org = df_draft.groupby('slugOrganizationTypeFrom')['idPlayer'].nunique().reset_index()
draft_player_org.columns = ['organization', 'player_count']

# Renommer les catégories
draft_player_org['organization'] = draft_player_org['organization'].replace({
    'COL': 'Université',
    'PRO': 'Autres équipes',
    'HS': 'Lycée',
    'UNK': 'Inconnu'
})

# Créer le graphique en secteurs (camembert)
fig_pie = px.pie(draft_player_org, values='player_count', names='organization', hole=0.3,
                 title="Répartition des joueurs draftés par organisation")

# Agréger les données nécessaires pour le graphique de ligne
draft_player_org_trend = df_draft.groupby(['yearDraft', 'slugOrganizationTypeFrom'])['idPlayer'].nunique().reset_index()
draft_player_org_trend.columns = ['year_drafted', 'organization', 'drafted_count']

# Renommer les catégories pour le graphique de ligne
draft_player_org_trend['organization'] = draft_player_org_trend['organization'].replace({
    'COL': 'Université',
    'PRO': 'Autres équipes',
    'HS': 'Lycée',
    'UNK': 'Inconnu'
})

# Créer le graphique de ligne
fig_line = px.line(draft_player_org_trend.dropna(), 
                   x="year_drafted", y="drafted_count", color="organization",
                   title="Tendance des drafts NBA par type d'organisation source de joueurs de 1949 à 2020")

# Mettre à jour les légendes en français
fig_line.update_layout(
    xaxis_title='Année de draft',
    yaxis_title='Nombre de joueurs draftés',
    title='Tendance des drafts NBA par source de joueurs de 1949 à 2020',
    legend_title='Organisation'
)

# Afficher les graphiques l'un à côté de l'autre
fig_pie.show()
fig_line.show()


## Evolution du jeu depuis 1946

### Taux de victoire à domicile

In [50]:
import pandas as pd
import plotly.express as px

# Charger les fichiers CSV
games = pd.read_csv('./csv/Game.csv')
teams = pd.read_csv('./csv/Team_Attributes.csv')

# Extraire l'année de la saison à partir de SEASON_ID
games['SEASON_YEAR'] = games['SEASON_ID'].astype(str).str[1:5].astype(int)

# Calculer le nombre de victoires et de matchs joués à domicile et à l'extérieur pour chaque équipe chaque saison
home_stats = games.groupby(['SEASON_YEAR', 'TEAM_ID_HOME']).agg(
    HOME_WINS=('WL_HOME', lambda x: (x == 'W').sum()),
    HOME_GAMES=('TEAM_ID_HOME', 'count')
).reset_index()
home_stats['HOME_WIN_PCT'] = (home_stats['HOME_WINS'] / home_stats['HOME_GAMES']) * 100
home_stats['game_location'] = 'home'
home_stats.rename(columns={'TEAM_ID_HOME': 'TEAM_ID'}, inplace=True)

away_stats = games.groupby(['SEASON_YEAR', 'TEAM_ID_AWAY']).agg(
    AWAY_WINS=('WL_AWAY', lambda x: (x == 'W').sum()),
    AWAY_GAMES=('TEAM_ID_AWAY', 'count')
).reset_index()
away_stats['AWAY_WIN_PCT'] = (away_stats['AWAY_WINS'] / away_stats['AWAY_GAMES']) * 100
away_stats['game_location'] = 'away'
away_stats.rename(columns={'TEAM_ID_AWAY': 'TEAM_ID'}, inplace=True)

# Fusionner les données de victoires à domicile et à l'extérieur
home_stats = home_stats[['SEASON_YEAR', 'TEAM_ID', 'HOME_WIN_PCT', 'game_location']]
home_stats.rename(columns={'HOME_WIN_PCT': 'win_percentage'}, inplace=True)
away_stats = away_stats[['SEASON_YEAR', 'TEAM_ID', 'AWAY_WIN_PCT', 'game_location']]
away_stats.rename(columns={'AWAY_WIN_PCT': 'win_percentage'}, inplace=True)

team_stats = pd.concat([home_stats, away_stats], ignore_index=True)

# Ajouter les noms des équipes en utilisant Team_Attributes.csv
teams = teams[['ID', 'NICKNAME']]
team_stats = pd.merge(team_stats, teams, left_on='TEAM_ID', right_on='ID', how='left')

# Calculer le pourcentage global de victoires à domicile pour chaque saison
overall_home_stats = games.groupby('SEASON_YEAR').agg(
    home_win_count=('WL_HOME', lambda x: (x == 'W').sum()),
    total_game_count=('TEAM_ID_HOME', 'count')
).reset_index()
overall_home_stats['home_won_percentage'] = (overall_home_stats['home_win_count'] / overall_home_stats['total_game_count']) * 100

# Utiliser plotly.express pour créer le graphique linéaire pour le pourcentage global de victoires à domicile
fig1 = px.line(overall_home_stats, x='SEASON_YEAR', y='home_won_percentage', 
               title='1946-2020: Pourcentage global de victoires à domicile',
               labels={'SEASON_YEAR': 'Saison', 'home_won_percentage': 'Pourcentage de victoires à domicile'})

fig1.show()

# Utiliser plotly.express pour créer le scatter plot
fig2 = px.scatter(team_stats, x='SEASON_YEAR', y='win_percentage', color='game_location', 
                 title='Pourcentage de victoires par équipe et par lieu de jeu (1946-2020)',
                 labels={'SEASON_YEAR': 'Saison', 'win_percentage': 'Pourcentage de victoires', 'game_location': 'Lieu de jeu', 'NICKNAME': 'Équipe'},
                 hover_data=['NICKNAME'])

fig2.show()


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_18068\4145886089.py:5: DtypeWarning:

Columns (58,64,65,68,79,93) have mixed types. Specify dtype option on import or set low_memory=False.



### Le tir à 3 points

In [52]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Charger le fichier CSV
df = pd.read_csv('./csv/Game.csv')

# Ajouter une colonne pour la saison (extraction de l'année)
df['season'] = df['GAME_DATE'].str[:4].astype(int)

# Exclure la saison 2021
df = df[df['season'] != 2021]

# Calculer le nombre de tirs à 3 points tentés et réussis par match pour chaque équipe
df['three_point_attempts_home'] = df['FG3A_HOME'].astype(float)
df['three_point_attempts_away'] = df['FG3A_AWAY'].astype(float)
df['three_point_made_home'] = df['FG3M_HOME'].astype(float)
df['three_point_made_away'] = df['FG3M_AWAY'].astype(float)

# Calculer le nombre de points par match pour chaque équipe
df['total_points'] = df['PTS_HOME'].astype(float) + df['PTS_AWAY'].astype(float)

# Agréger les données par saison pour obtenir la moyenne des tirs à 3 points tentés et réussis par match et la moyenne des points par match
season_stats = df.groupby('season').agg({
    'three_point_attempts_home': 'mean', 
    'three_point_attempts_away': 'mean', 
    'three_point_made_home': 'mean', 
    'three_point_made_away': 'mean', 
    'total_points': 'mean'
}).reset_index()

season_stats['avg_3P_attempts'] = season_stats[['three_point_attempts_home', 'three_point_attempts_away']].mean(axis=1)
season_stats['avg_3P_made'] = season_stats[['three_point_made_home', 'three_point_made_away']].mean(axis=1)

# Filtrer les données pour afficher les saisons après 1986
season_stats_filtered = season_stats[season_stats['season'] > 1986]

# Créer une visualisation avec deux axes y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Tracer le nombre moyen de tirs à 3 points tentés par match
trace_3P_attempts = go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['avg_3P_attempts'], 
    mode='lines', 
    name='Nombre de tirs à 3 points tentés par match', 
    line=dict(color='blue')
)
fig.add_trace(trace_3P_attempts, secondary_y=False)

# Tracer le nombre moyen de tirs à 3 points réussis par match
trace_3P_made = go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['avg_3P_made'], 
    mode='lines', 
    name='Nombre de tirs à 3 points réussis par match', 
    line=dict(color='red')
)
fig.add_trace(trace_3P_made, secondary_y=False)

# Tracer le nombre moyen de points par match
trace_points = go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['total_points'], 
    mode='lines', 
    name='Points par match', 
    line=dict(color='green')
)
fig.add_trace(trace_points, secondary_y=True)

# Mettre à jour les titres et les axes
fig.update_yaxes(title_text="Nombre de tirs à 3 points par match", secondary_y=False)
fig.update_yaxes(title_text="Points par match", secondary_y=True)
fig.update_layout(title_text="Évolution du nombre de tirs à 3 points tentés et réussis par match et de la moyenne des points par match en NBA depuis 1986")

fig.show()


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_18068\692667930.py:7: DtypeWarning:

Columns (58,64,65,68,79,93) have mixed types. Specify dtype option on import or set low_memory=False.



#### Tir à 2 points VS 3 points

In [53]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Charger le fichier CSV
df = pd.read_csv('./csv/Game.csv')

# Ajouter une colonne pour la saison (extraction de l'année)
df['season'] = df['GAME_DATE'].str[:4].astype(int)

# Exclure la saison 2021
df = df[df['season'] != 2021]

# Calculer le pourcentage de tirs à 3 points pour chaque équipe et chaque saison
df['FG3_PCT_HOME'] = df['FG3M_HOME'].astype(float) / df['FG3A_HOME'].astype(float)
df['FG3_PCT_AWAY'] = df['FG3M_AWAY'].astype(float) / df['FG3A_AWAY'].astype(float)

# Calculer le pourcentage de tirs à 2 points pour chaque équipe et chaque saison
df['FG2M_HOME'] = df['FGM_HOME'].astype(float) - df['FG3M_HOME'].astype(float)
df['FG2A_HOME'] = df['FGA_HOME'].astype(float) - df['FG3A_HOME'].astype(float)
df['FG2M_AWAY'] = df['FGM_AWAY'].astype(float) - df['FG3M_AWAY'].astype(float)
df['FG2A_AWAY'] = df['FGA_AWAY'].astype(float) - df['FG3A_AWAY'].astype(float)
df['FG2_PCT_HOME'] = df['FG2M_HOME'] / df['FG2A_HOME']
df['FG2_PCT_AWAY'] = df['FG2M_AWAY'] / df['FG2A_AWAY']

# Agréger les données par saison pour obtenir les moyennes des pourcentages
season_stats = df.groupby('season').agg({
    'FG3_PCT_HOME': 'mean', 'FG3_PCT_AWAY': 'mean',
    'FG2_PCT_HOME': 'mean', 'FG2_PCT_AWAY': 'mean'
}).reset_index()

# Calculer les moyennes globales des pourcentages
season_stats['avg_3P_pct'] = season_stats[['FG3_PCT_HOME', 'FG3_PCT_AWAY']].mean(axis=1)
season_stats['avg_2P_pct'] = season_stats[['FG2_PCT_HOME', 'FG2_PCT_AWAY']].mean(axis=1)

# Calculer les points moyens obtenus par tir à 3 points et à 2 points
season_stats['points_per_3P'] = 3 * season_stats['avg_3P_pct']
season_stats['points_per_2P'] = 2 * season_stats['avg_2P_pct']

# Filtrer les données pour afficher les saisons après 1986
season_stats_filtered = season_stats[season_stats['season'] > 1986]

# Créer une visualisation avec deux axes y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Tracer les points moyens obtenus par tir à 3 points
trace_3P = go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['points_per_3P'], 
    mode='lines', 
    name='Points par tir à 3 points', 
    line=dict(color='blue')
)
fig.add_trace(trace_3P, secondary_y=False)

# Tracer les points moyens obtenus par tir à 2 points
trace_2P = go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['points_per_2P'], 
    mode='lines', 
    name='Points par tir à 2 points', 
    line=dict(color='green')
)
fig.add_trace(trace_2P, secondary_y=False)

# Ajouter des annotations pour indiquer les pourcentages de réussite
fig.add_trace(go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['avg_3P_pct'], 
    mode='markers+text', 
    name='Pourcentage de tirs à 3 points',
    text=[f"{pct:.2%}" for pct in season_stats_filtered['avg_3P_pct']],
    textposition="top center",
    marker=dict(color='blue')
), secondary_y=True)

fig.add_trace(go.Scatter(
    x=season_stats_filtered['season'], 
    y=season_stats_filtered['avg_2P_pct'], 
    mode='markers+text', 
    name='Pourcentage de tirs à 2 points',
    text=[f"{pct:.2%}" for pct in season_stats_filtered['avg_2P_pct']],
    textposition="bottom center",
    marker=dict(color='green')
), secondary_y=True)

# Mettre à jour les titres et les axes
fig.update_yaxes(title_text="Points par tir", secondary_y=False)
fig.update_yaxes(title_text="Pourcentage de réussite", secondary_y=True)
fig.update_layout(
    title_text="Points moyens obtenus par tir à 3 points et à 2 points en NBA depuis 1986",
    legend_title_text="Type de tir"
)

fig.show()


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6732\733209220.py:7: DtypeWarning:

Columns (58,64,65,68,79,93) have mixed types. Specify dtype option on import or set low_memory=False.



En 2020, pour que le tir à 2 points soit plus efficace, il devrait dépasser les 54.12% de réussite.

## Les salaires

In [66]:
import pandas as pd

# Charger les fichiers CSV (remplacez les chemins par les chemins réels de vos fichiers CSV)
player_salary = pd.read_csv('./csv/Player_Salary.csv')
player = pd.read_csv('./csv/Player.csv')
player_attributes = pd.read_csv('./csv/Player_Attributes.csv')

# Simuler les données de la table Player_Salary
player_salary = player_salary.rename(columns={'namePlayer': 'player_name', 'nameTeam': 'team_name', 'value': 'salary', 'slugSeason': 'season'})
player_salary['season'] = '2020-21'

# Simuler les données de la table Player
player = player.rename(columns={'id': 'player_id', 'full_name': 'player_name'})

# Simuler les données de la table Player_Attributes
player_attributes = player_attributes.rename(columns={'ID': 'player_id', 'BIRTHDATE': 'birthdate', 'DRAFT_YEAR': 'draft_year', 'POSITION': 'position'})

# Convertir les types de données
player_attributes['birthdate'] = pd.to_datetime(player_attributes['birthdate'], errors='coerce')
player_attributes['draft_year'] = pd.to_numeric(player_attributes['draft_year'], errors='coerce')

# Joindre les trois tables
merged_data = player_salary.merge(player, on='player_name', how='left').merge(player_attributes, on='player_id', how='left')

# Calculer les colonnes supplémentaires
merged_data['age'] = (2021 - merged_data['birthdate'].dt.year).astype('Int64')
merged_data['years_in_NBA'] = (2021 - merged_data['draft_year']).astype('Int64')
merged_data['draft_year'] = merged_data['draft_year'].astype('Int64')  # Convertir en entier
merged_data['salary_in_millions'] = round(merged_data['salary'] / 1000000, 2)

# Filtrer les données pour la saison 2020-21 et trier par salaire
player_salary_top_10 = merged_data[merged_data['season'] == '2020-21'].sort_values(by='salary_in_millions', ascending=False)

# Exclure les joueurs qui sont présents plusieurs fois
player_salary_top_10 = player_salary_top_10.drop_duplicates(subset=['player_name']).head(10)

# Sélectionner les colonnes nécessaires
player_salary_top_10 = player_salary_top_10[['player_name', 'team_name', 'age', 'draft_year', 'years_in_NBA', 'position', 'salary_in_millions']]

# Afficher les données dans un joli tableau
player_salary_top_10.rename(columns={
    'player_name': 'Nom du Joueur',
    'team_name': 'Équipe',
    'age': 'Âge',
    'draft_year': 'Année de Draft',
    'years_in_NBA': 'Années en NBA',
    'position': 'Position',
    'salary_in_millions': 'Salaire (en millions)'
}, inplace=True)

player_salary_top_10


,Nom du Joueur,Équipe,Âge,Année de Draft,Années en NBA,Position,Salaire (en millions)
696,Giannis Antetokounmpo,Milwaukee Bucks,27,2013,8,Forward,48.79
1057,Damian Lillard,Portland Trail Blazers,31,2012,9,Guard,48.79
547,Paul George,Los Angeles Clippers,31,2010,11,Forward,48.79
106,James Harden,Brooklyn Nets,32,2009,12,Guard,47.37
465,John Wall,Houston Rockets,31,2010,11,Guard,47.37
1289,Russell Westbrook,Washington Wizards,33,2008,13,Guard,47.06
432,Stephen Curry,Golden State Warriors,33,2009,12,Guard,45.78
582,LeBron James,Los Angeles Lakers,37,2003,18,Forward,44.47
1007,Chris Paul,Phoenix Suns,36,2005,16,Guard,44.21
114,Kevin Durant,Brooklyn Nets,33,2007,14,Forward,43.93


### Les postes les mieux payés

In [5]:
import pandas as pd
import plotly.express as px

# Charger les fichiers CSV (remplacez les chemins par les chemins réels de vos fichiers CSV)
player_salary = pd.read_csv('./csv/Player_Salary.csv')
player = pd.read_csv('./csv/Player.csv')
player_attributes = pd.read_csv('./csv/Player_Attributes.csv')

# Simuler les données de la table Player_Salary
player_salary = player_salary.rename(columns={'namePlayer': 'player_name', 'nameTeam': 'team_name', 'value': 'salary', 'slugSeason': 'season'})
player_salary['season'] = '2020-21'

# Simuler les données de la table Player
player = player.rename(columns={'id': 'player_id', 'full_name': 'player_name'})

# Simuler les données de la table Player_Attributes
player_attributes = player_attributes.rename(columns={'ID': 'player_id', 'BIRTHDATE': 'birthdate', 'DRAFT_YEAR': 'draft_year', 'POSITION': 'position'})

# Convertir les types de données
player_attributes['birthdate'] = pd.to_datetime(player_attributes['birthdate'], errors='coerce')
player_attributes['draft_year'] = pd.to_numeric(player_attributes['draft_year'], errors='coerce')

# Filtrer les joueurs actifs
player_active = player[player['is_active'] == 1]

# Joindre les trois tables
merged_data = player_salary.merge(player_active, on='player_name', how='left').merge(player_attributes, on='player_id', how='left')

# Filtrer les données pour la saison 2020-21
merged_data_2020 = merged_data[merged_data['season'] == '2020-21']

# Calculer les colonnes supplémentaires
merged_data_2020['salary_in_millions'] = round(merged_data_2020['salary'] / 1000000, 2)

# Calculer la moyenne des salaires par poste
avg_salary_by_position = merged_data_2020.groupby('position')['salary_in_millions'].mean().reset_index()

# Créer l'histogramme
fig = px.bar(avg_salary_by_position, x='position', y='salary_in_millions',
             title='Moyenne de salaire par poste pour les joueurs actifs en 2020',
             labels={'position': 'Poste', 'salary_in_millions': 'Salaire (en millions)'})

fig.show()


### La corrélation entre les statistiques d'un joueur et son salaire

In [36]:
import pandas as pd
import plotly.express as px

# Charger les fichiers CSV
players = pd.read_csv('./csv/Player.csv')
salaries = pd.read_csv('./csv/Player_Salary.csv')
attributes = pd.read_csv('./csv/Player_Attributes.csv')

# Filtrer les joueurs actifs
active_players = players[players['is_active'] == 1]

# Filtrer les salaires pour l'année 2020
salaries_2020 = salaries[salaries['slugSeason'] == '2020-21']

# Fusionner les données des joueurs actifs avec leurs attributs et salaires
active_players_2020 = active_players.merge(attributes, left_on=['full_name'], right_on=['DISPLAY_FIRST_LAST'])
active_players_2020 = active_players_2020.merge(salaries_2020, left_on=['full_name'], right_on=['namePlayer'])

# Extraire les salaires, les points, les assists et les rebonds
salary_points_2020 = active_players_2020[['full_name', 'value', 'PTS', 'AST', 'REB']]

# Créer un bubble chart interactif
fig = px.scatter(salary_points_2020, x='value', y='PTS', size='REB', color='AST', hover_data=['full_name'],
                 labels={'value': 'Salaire ($)', 'PTS': 'Points par match', 'REB': 'Rebonds', 'AST': 'Assists'},
                 title='Corrélation entre le salaire, les points, les assists et les rebonds des joueurs actifs en 2020')

fig.show()
